## Spotify Web API data


In [12]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
import numpy as np
import matplotlib

In [13]:
from dotenv import load_dotenv
import os
import base64
from requests import post
import json

load_dotenv()

client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")
REDIRECT_URI = "https://localhost/"


def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")
    
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}
token = get_token()

sp_oauth = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri = REDIRECT_URI, scope = "user-library-read")

access_token = sp_oauth.get_access_token()

sp = spotipy.Spotify(auth_manager=sp_oauth)

C:\Users\justi\AppData\Local\Temp\ipykernel_41672\2948581706.py:36: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  access_token = sp_oauth.get_access_token()


In [72]:
playlist_ids = [
    '37i9dQZF1DWUZv12GM5cFk',
    '37i9dQZF1DX9Ol4tZWPH6V',
    '37i9dQZF1DX0P7PzzKwEKl',
    '37i9dQZF1DXaW8fzPh9b08',
    '37i9dQZF1DWTWdbR13PQYH',
    '37i9dQZF1DWWzQTBs5BHX9',
    '37i9dQZF1DX1vSJnMeoy3V',
    '37i9dQZF1DX3j9EYdzv2N9',
    '37i9dQZF1DWYuGZUE4XQXm',
    '37i9dQZF1DX4UkKv8ED8jp',
    '37i9dQZF1DXc6IFF23C9jj'
]
all_tracks = []
for id in playlist_ids:
    tracks = sp.playlist(id)['tracks']['items']
    for track in tracks:
        year = int(track['track']['album']['release_date'][:4])
        if year >= 2000 and year <= 2010:
            all_tracks.append(track['track'])

In [73]:
# ALL CATEGORIES FOR TRACKS
# album (get release date)
# artists (get the )
# available_markets (GET THE LENGTH)
# duration_ms (DIVIDE BY 1000)
# explicit
# name
# popularity

#MAKE SURE TO REMOVE DUPLICATES

211.16

In [195]:
df = pd.DataFrame(all_tracks)
df = df[["album", "artists", "available_markets", "duration_ms", "explicit", "name", "popularity"]]
for i in np.arange(len(df["artists"])):
    df["artists"][i] = df["artists"][i][0]["name"]
    df["album"][i] = int(df["album"][i]["release_date"][:4])
df.rename(columns = {"album": "year"})

/var/folders/kg/5fvpp0xn4cx3m6fbhrk9hnkc0000gn/T/ipykernel_13336/424785257.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["artists"][i] = df["artists"][i][0]["name"]
/var/folders/kg/5fvpp0xn4cx3m6fbhrk9hnkc0000gn/T/ipykernel_13336/424785257.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["album"][i] = int(df["album"][i]["release_date"][:4])


,year,artists,available_markets,duration_ms,explicit,name,popularity
0,2000,Britney Spears,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",211160,False,Oops!...I Did It Again,80
1,2000,Bon Jovi,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",224493,False,It's My Life,81
2,2000,*NSYNC,"[AR, BO, BR, CA, CL, CO, CR, DO, EC, SV, GT, H...",200400,False,Bye Bye Bye,74
3,2000,Eminem,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",284200,True,The Real Slim Shady,87
4,2001,Modjo,"[AR, AU, BE, BO, BR, BG, CA, CL, CO, CR, CY, C...",307153,False,Lady - Hear Me Tonight,78
...,...,...,...,...,...,...,...
1019,2010,Shontelle,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",226533,False,Impossible,68
1020,2009,David Guetta,[US],188000,False,Gettin' Over You (feat. Fergie & LMFAO),50
1021,2010,Tinie Tempah,"[CA, MX, US]",207600,False,Written in the Stars (feat. Eric Turner),46
1022,2010,Sara Bareilles,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",207493,False,King of Anything,61
